In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('data/events.csv', low_memory=False)
df['timestamp'] = pd.to_datetime(df['timestamp'])

In [3]:
df.head()

,timestamp,event,person,url,sku,model,condition,storage,color,skus,...,search_engine,channel,new_vs_returning,city,region,country,device_type,screen_resolution,operating_system_version,browser_version
0,2018-05-18 00:11:59,viewed product,4886f805,NaN,9288.0,Samsung Galaxy J7 Prime,Excelente,32GB,Dourado,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2018-05-18 00:11:27,viewed product,ad93850f,NaN,304.0,iPhone 5s,Muito Bom,32GB,Cinza espacial,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2018-05-18 00:11:16,viewed product,0297fc1e,NaN,6888.0,iPhone 6S,Muito Bom,64GB,Prateado,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2018-05-18 00:11:14,viewed product,2d681dd8,NaN,11890.0,iPhone 7,Bom,128GB,Vermelho,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2018-05-18 00:11:09,viewed product,cccea85e,NaN,7517.0,LG G4 H818P,Excelente,32GB,Branco,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
df['days_elapsed'] = pd.to_datetime('2018-06-01') - df['timestamp']
df['days_elapsed'] = df['days_elapsed'] / np.timedelta64(1, 'D')

In [20]:
events = df[['person', 'event', 'days_elapsed']]

In [21]:
events.head()

,person,event,days_elapsed
0,4886f805,viewed product,13.991678
1,ad93850f,viewed product,13.992049
2,0297fc1e,viewed product,13.992176
3,2d681dd8,viewed product,13.992199
4,cccea85e,viewed product,13.992257


#### Ahora voy a filtrar los eventos para los ultimos 15 dias.

In [22]:
max_days = 15
min_days = 0
events = events.loc[(events['days_elapsed'] < max_days) & (events['days_elapsed'] > min_days), :]

In [23]:
events = events.groupby(['person', 'event']).agg({'days_elapsed':'count'})

In [24]:
events = events.reset_index()

In [27]:
events = events.pivot_table(index='person', columns='event', values='days_elapsed')

In [30]:
events = events.fillna(0)

In [37]:
events.head(7)

event,ad campaign hit,brand listing,checkout,conversion,generic listing,lead,search engine hit,searched products,staticpage,viewed product,visited site
person,,,,,,,,,,,
0008ed71,0.0,0.0,3.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,2.0
00091926,11.0,3.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,229.0,15.0
000ba417,1.0,24.0,6.0,1.0,14.0,0.0,1.0,0.0,0.0,153.0,6.0
000c79fe,1.0,0.0,1.0,0.0,1.0,0.0,1.0,9.0,0.0,3.0,1.0
000e4d9e,3.0,14.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,186.0,10.0
000e619d,4.0,9.0,1.0,0.0,3.0,0.0,0.0,6.0,0.0,23.0,3.0
001001be,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0


In [38]:
string = '_in_last_15_days'

events.columns = [
    'ad_hits' + string,
    'brand_listings' + string,
    'checkouts' + string,
    'conversions' + string,
    'generic_listings' + string,
    'leads' + string,
    'search_engine_hits' + string,
    'searched_products' + string,
    'static_page' + string,
    'viewed_products' + string,
    'visited_site' + string
]

In [40]:
events.to_csv('data/eventos_en_los_ultimos_15_dias.csv')  # si cambian los dias, cambien el nombre